In [20]:
import os

# 1. THAY THẾ 'đường_dẫn_tới_thư_mục_color' BẰNG ĐƯỜNG DẪN THỰC TẾ CỦA BẠN.
# Dựa trên hình ảnh, có thể là: 'D:\Plant Villages\plantvillage dataset\color'
# (Bạn cần điều chỉnh ký tự phân cách tùy thuộc vào hệ điều hành,
# hoặc dùng os.path.join() cho tính tương thích).
base_dir = r"D:\plant_villages\color" # Ví dụ: r"D:\Long (ID)\Plant Villages\plantvillage dataset\color"

# Dictionary để lưu kết quả đếm
counts = {}

# Lặp qua tất cả các mục (thư mục) trong thư mục cơ sở
for entry in os.listdir(base_dir):
    # Tạo đường dẫn đầy đủ đến thư mục con
    folder_path = os.path.join(base_dir, entry)

    # Chỉ xử lý các thư mục (không phải file)
    if os.path.isdir(folder_path):
        # Đếm số lượng file trong thư mục con này
        # Lấy danh sách tất cả các mục trong folder_path
        # và lọc ra chỉ những mục là tệp (file)
        file_count = sum(
            1
            for item in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, item))
        )

        # Lưu kết quả
        counts[entry] = file_count

# In ra kết quả
print("--- Số lượng ảnh trong mỗi thư mục ---")
for folder_name, count in counts.items():
    print(f"{folder_name}: {count}")

print("\n--- Tổng số ảnh trong toàn bộ dataset ---")
total_images = sum(counts.values())
print(f"Tổng cộng: {total_images} ảnh")

--- Số lượng ảnh trong mỗi thư mục ---
Apple___Apple_scab: 630
Apple___Black_rot: 621
Apple___Cedar_apple_rust: 275
Apple___healthy: 1645
Blueberry___healthy: 1502
Cherry_(including_sour)___healthy: 854
Cherry_(including_sour)___Powdery_mildew: 1052
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 513
Corn_(maize)___Common_rust: 1192
Corn_(maize)___healthy: 1162
Corn_(maize)___Northern_Leaf_Blight: 985
Grape___Black_rot: 1180
Grape___Esca_(Black_Measles): 1383
Grape___healthy: 1260
Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1076
Orange___Haunglongbing_(Citrus_greening): 5507
Peach___Bacterial_spot: 2297
Peach___healthy: 360
Pepper,_bell___Bacterial_spot: 997
Pepper,_bell___healthy: 1478
Potato___Early_blight: 1000
Potato___healthy: 1169
Potato___Late_blight: 1000
Raspberry___healthy: 371
Soybean___healthy: 5090
Squash___Powdery_mildew: 1835
Strawberry___healthy: 456
Strawberry___Leaf_scorch: 1109
Tomato___Bacterial_spot: 2127
Tomato___Early_blight: 1000
Tomato___healthy: 1591
Tomat

In [3]:
pip install numpy Pillow scikit-learn albumentations


   ------------------------------ --------- 3/4 [albumentations]
   ------------------------------ --------- 3/4 [albumentations]
   ---------------------------------------- 4/4 [albumentations]

Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import shutil
import random
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import albumentations as A

# --- CẤU HÌNH ---
# ⚠️ THAY ĐỔI ĐƯỜNG DẪN NÀY ĐẾN THƯ MỤC CHỨA DỮ LIỆU GỐC (VD: 'color')
BASE_DATA_DIR = r"D:\plant_villages\color" 
OUTPUT_DIR = r"D:\plant_villages\sampled" # Thư mục chứa kết quả

TARGET_TRAIN_COUNT = 500
TARGET_TEST_COUNT = 100

# Danh sách các lớp (thư mục) cần xử lý
CLASSES_TO_PROCESS = {
    "Apple": ["Apple___Apple_scab", "Apple___Black_rot", "Apple___healthy"],
    "Corn (Maize)": ["Corn_(maize)___Common_rust", "Corn_(maize)___Northern_Leaf_Blight", "Corn_(maize)___healthy"],
    "Tomato": ["Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___healthy"],
    "Potato": ["Potato___Early_blight", "Potato___Late_blight", "Potato___healthy"],
    "Grape": ["Grape___Black_rot", "Grape___Esca_(Black_Measles)", "Grape___healthy"],
}

# Định nghĩa các phép Tăng cường dữ liệu (Chỉ dùng cho TẬP TRAIN)
augmentor = A.Compose([
    A.HorizontalFlip(p=0.5), 
    A.VerticalFlip(p=0.2),   
    # Dùng Affine thay cho ShiftScaleRotate
    A.Affine(
        scale=(0.8, 1.2), # tương đương scale_limit=0.2
        translate_percent=(0.0625, 0.0625), # tương đương shift_limit=0.0625
        rotate=(-20, 20), # tương đương rotate_limit=20
        shear=0, 
        p=0.7
    ),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(p=0.2),
    A.Resize(224, 224) 
])

# --- HÀM HỖ TRỢ ---

def get_image_files(folder_path):
    """Lấy danh sách đầy đủ đường dẫn đến các file ảnh (jpg, png) trong thư mục."""
    image_files = []
    for f in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_files.append(os.path.join(folder_path, f))
    return image_files

def augment_and_save(original_path, output_dir, class_name, index):
    """Thực hiện Data Augmentation và lưu ảnh mới."""
    try:
        # 1. Đọc ảnh
        image = np.array(Image.open(original_path).convert("RGB"))
        
        # 2. Augmentation
        augmented = augmentor(image=image)
        augmented_image = augmented['image']
        
        # 3. Lưu ảnh
        output_filename = f"{class_name}_{index:05d}_aug.jpg"
        output_path = os.path.join(output_dir, output_filename)
        Image.fromarray(augmented_image).save(output_path)
        return True
    except Exception as e:
        # print(f"Lỗi khi augment ảnh {original_path}: {e}") # Có thể bỏ comment để debug
        return False

def copy_and_rename_file(source_path, dest_dir, class_name, index):
    """Sao chép và đổi tên file ảnh."""
    try:
        ext = os.path.splitext(source_path)[1].lower()
        # Định dạng tên file mới: classname_index.ext
        new_filename = f"{class_name}_{index:05d}{ext}"
        dest_path = os.path.join(dest_dir, new_filename)
        shutil.copy2(source_path, dest_path)
        return True
    except Exception as e:
        # print(f"Lỗi khi sao chép ảnh {source_path}: {e}") # Có thể bỏ comment để debug
        return False

# --- QUY TRÌNH CHÍNH ---

def process_dataset():
    """Thực hiện toàn bộ quy trình."""
    print("--- BẮT ĐẦU XỬ LÝ DỮ LIỆU ---")
    
    # Tạo thư mục đầu ra
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "train"), exist_ok=True)
    os.makedirs(os.path.join(OUTPUT_DIR, "test"), exist_ok=True)

    # Lặp qua từng lớp cần lấy
    for genus, classes in CLASSES_TO_PROCESS.items():
        for class_name in classes:
            print(f"\n✅ Đang xử lý lớp: {class_name}")
            
            # Tên lớp chuẩn hóa cho việc đặt tên file (ví dụ: Apple_Apple_scab)
            safe_class_name = class_name.replace("__", "_")

            # 1. Lấy danh sách file gốc
            source_folder = os.path.join(BASE_DATA_DIR, class_name)
            if not os.path.exists(source_folder):
                print(f"❌ CẢNH BÁO: Thư mục nguồn không tồn tại: {source_folder}")
                continue
                
            all_files = get_image_files(source_folder)
            random.shuffle(all_files) # Trộn ngẫu nhiên danh sách file
            num_total = len(all_files)
            print(f"   -> Tổng số ảnh gốc: {num_total}")

            # 2. XỬ LÝ TẬP TRAIN (500 ảnh, có Augmentation nếu thiếu)

            # Lấy các ảnh gốc cho tập Train (tối đa 500)
            train_pool = all_files[:TARGET_TRAIN_COUNT]
            num_train_current = len(train_pool)
            print(f"   -> Số ảnh train gốc được lấy: {num_train_current}")

            train_output_dir = os.path.join(OUTPUT_DIR, "train", class_name)
            os.makedirs(train_output_dir, exist_ok=True)
            
            # Sao chép các ảnh Train gốc
            for i, file_path in enumerate(train_pool):
                copy_and_rename_file(file_path, train_output_dir, safe_class_name, i)

            # --- Data Augmentation cho tập Train nếu thiếu ---
            missing_train = TARGET_TRAIN_COUNT - num_train_current
            if missing_train > 0:
                print(f"   -> Thiếu {missing_train} ảnh Train. Bắt đầu Augmentation...")
                # Lấy ngẫu nhiên các ảnh hiện có để tăng cường
                # Dùng random.choices để có thể chọn lại ảnh đã dùng
                augment_candidates = random.choices(train_pool, k=missing_train)
                
                start_index = num_train_current
                current_aug_count = 0
                for i in range(missing_train):
                    file_to_augment = augment_candidates[i]
                    if augment_and_save(file_to_augment, train_output_dir, safe_class_name, start_index + i):
                        current_aug_count += 1
                
                print(f"   -> Đã tạo thêm {current_aug_count} ảnh Train bằng Augmentation. Tổng Train: {num_train_current + current_aug_count}")
            else:
                 print(f"   -> Đã đủ 500 ảnh Train gốc.")

            # 3. XỬ LÝ TẬP TEST (Tối đa 100 ảnh, CHỈ dùng ảnh gốc)

            # Lấy các ảnh còn lại sau khi đã lấy 500 ảnh đầu tiên cho tập Train
            remaining_files = all_files[TARGET_TRAIN_COUNT:] 
            
            # Lấy tối đa 100 ảnh từ phần còn lại
            test_pool = remaining_files[:TARGET_TEST_COUNT]
            num_test_current = len(test_pool)
            
            test_output_dir = os.path.join(OUTPUT_DIR, "test", class_name)
            os.makedirs(test_output_dir, exist_ok=True)
            
            # Sao chép tập Test hiện có
            for i, file_path in enumerate(test_pool):
                copy_and_rename_file(file_path, test_output_dir, safe_class_name, i)
            
            print(f"   -> Đã lấy {num_test_current} ảnh GỐC cho tập Test.")

    print("\n--- HOÀN TẤT ---")
    print(f"Dữ liệu đã được lưu vào thư mục: {OUTPUT_DIR}")

In [22]:
if __name__ == "__main__":
    process_dataset()

--- BẮT ĐẦU XỬ LÝ DỮ LIỆU ---

✅ Đang xử lý lớp: Apple___Apple_scab
   -> Tổng số ảnh gốc: 630
   -> Số ảnh train gốc được lấy: 500
   -> Đã đủ 500 ảnh Train gốc.
   -> Đã lấy 100 ảnh GỐC cho tập Test.

✅ Đang xử lý lớp: Apple___Black_rot
   -> Tổng số ảnh gốc: 621
   -> Số ảnh train gốc được lấy: 500
   -> Đã đủ 500 ảnh Train gốc.
   -> Đã lấy 100 ảnh GỐC cho tập Test.

✅ Đang xử lý lớp: Apple___healthy
   -> Tổng số ảnh gốc: 1645
   -> Số ảnh train gốc được lấy: 500
   -> Đã đủ 500 ảnh Train gốc.
   -> Đã lấy 100 ảnh GỐC cho tập Test.

✅ Đang xử lý lớp: Corn_(maize)___Common_rust
   -> Tổng số ảnh gốc: 1192
   -> Số ảnh train gốc được lấy: 500
   -> Đã đủ 500 ảnh Train gốc.
   -> Đã lấy 100 ảnh GỐC cho tập Test.

✅ Đang xử lý lớp: Corn_(maize)___Northern_Leaf_Blight
   -> Tổng số ảnh gốc: 985
   -> Số ảnh train gốc được lấy: 500
   -> Đã đủ 500 ảnh Train gốc.
   -> Đã lấy 100 ảnh GỐC cho tập Test.

✅ Đang xử lý lớp: Corn_(maize)___healthy
   -> Tổng số ảnh gốc: 1162
   -> Số ảnh trai

Train AI

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# ==============================
# 1️⃣ CẤU HÌNH
# ==============================
train_dir = r"D:\plant_villages\sampled\train"
test_dir = r"D:\plant_villages\sampled\test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 40  # tăng lên
AUTOTUNE = tf.data.AUTOTUNE

# ==============================
# 2️⃣ LOAD DỮ LIỆU
# ==============================
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"Số lớp: {num_classes}")
print("Danh sách lớp:", class_names)

# ==============================
# 3️⃣ DATA AUGMENTATION
# ==============================
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.25),
    layers.RandomZoom(0.25),
    layers.RandomContrast(0.25),
    layers.RandomBrightness(0.25),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x), y))
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ==============================
# 4️⃣ MODEL
# ==============================
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = True  # mở hết layer để fine-tune sâu

for layer in base_model.layers[:50]:  # chỉ khóa 50 layer đầu
    layer.trainable = False

inputs = keras.Input(shape=IMG_SIZE + (3,))
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training=True)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# ==============================
# 5️⃣ CALLBACKS
# ==============================
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", monitor="val_accuracy", save_best_only=True, mode="max"
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=6, restore_best_weights=True
    )
]

# ==============================
# 6️⃣ TRAIN
# ==============================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)

# ==============================
# 7️⃣ LƯU & CONVERT
# ==============================
model.save("plant_model_final.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("plant_model.tflite", "wb") as f:
    f.write(tflite_model)

with open("labels.txt", "w", encoding="utf-8") as f:
    for label in class_names:
        f.write(label + "\n")

print("✅ Đã train & xuất model thành công!")




Found 7500 files belonging to 15 classes.
Found 1500 files belonging to 15 classes.
Số lớp: 15
Danh sách lớp: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___healthy', 'Corn_(maize)___Common_rust', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_1 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │        19,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,277,199 (8.69 MB)

 Trainable params: 2,193,807 (8.37 MB)

...

235/235 ━━━━━━━━━━━━━━━━━━━━ 341s 1s/step - accuracy: 0.9888 - loss: 0.0500 - val_accuracy: 0.7940 - val_loss: 0.5996
Epoch 3/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9991 - loss: 0.0129  

235/235 ━━━━━━━━━━━━━━━━━━━━ 347s 1s/step - accuracy: 0.9991 - loss: 0.0120 - val_accuracy: 0.8860 - val_loss: 0.3395
Epoch 4/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0052  

235/235 ━━━━━━━━━━━━━━━━━━━━ 342s 1s/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 0.9140 - val_loss: 0.2429
Epoch 5/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0024  

235/235 ━━━━━━━━━━━━━━━━━━━━ 342s 1s/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.9367 - val_loss: 0.1858
Epoch 6/40
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 0.0018  

235/235 ━━━━━━━━━━━━━━━━━━━━ 369s 2s/step - accuracy: 1.0000 - loss: 0.0017 - val_accuracy: 0.9493 - val_loss: 0.1539
Epoch 7/40
 13/235 ━━━━━━━━━━━━━━━━━━━━ 5:07 1s/step - accuracy: 1.0000 - loss: 0.0016